In [55]:
# # Data Cleaning for FinMark Corporation
# **Objective**: Clean event logs, trend reports, and marketing data to support:
# 1. Order pattern analysis
# 2. Peak time/downtime correlation
# 3. Inventory demand forecasting

In [57]:
# Steps:
# 1. Show null value summary before cleaning
# 2. Remove undefined columns (`col_6` to `col_50`)
# 3. Handle missing values in key columns
# 4. Ensure correct data types
# 5. Show null value summary after cleaning

In [59]:
import pandas as pd
import numpy as np

In [61]:
# 1. Event Logs Cleaning
# - **Critical Columns**: `user_id`, `event_type`, `event_time`, `product_id`, `amount`

In [63]:
# Load data
event_logs = pd.read_csv('event_logs.csv')

In [65]:
# Show initial null summary
print("=== Event Logs - BEFORE CLEANING ===")
print(f"Total records: {len(event_logs)}")
print("Null values per column:")
print(event_logs.iloc[:, :5].isnull().sum())

=== Event Logs - BEFORE CLEANING ===
Total records: 2000
Null values per column:
user_id          0
event_type       0
event_time       0
product_id       0
amount        1016
dtype: int64


In [67]:
# Remove undefined columns
event_logs_clean = event_logs.iloc[:, :5].copy()

In [69]:
# Rename columns for clarity
event_logs_clean.columns = ['user_id', 'event_type', 'event_time', 'product_id', 'amount']

In [71]:
# Convert to datetime - specify common formats to avoid warnings
event_logs_clean['event_time'] = pd.to_datetime(
    event_logs_clean['event_time'], 
    format='mixed',  # Handle multiple datetime formats
    errors='coerce'
)

In [73]:
# Handle missing values
event_logs_clean['event_type'] = event_logs_clean['event_type'].fillna('unknown')
event_logs_clean['user_id'] = event_logs_clean['user_id'].fillna('UNK').astype(str)

In [75]:
# Product/amount handling: Only relevant for orders
order_mask = event_logs_clean['event_type'].str.contains('order|purchase', case=False, na=False)
event_logs_clean.loc[order_mask, 'product_id'] = event_logs_clean.loc[order_mask, 'product_id'].fillna('PROD_UNK')
event_logs_clean.loc[order_mask, 'amount'] = event_logs_clean.loc[order_mask, 'amount'].fillna(0)
event_logs_clean.loc[~order_mask, 'product_id'] = event_logs_clean.loc[~order_mask, 'product_id'].fillna('N/A')
event_logs_clean.loc[~order_mask, 'amount'] = 0

In [77]:
# Show post-cleaning summary
print("\n=== Event Logs - AFTER CLEANING ===")
print(f"Total records: {len(event_logs_clean)}")
print("Null values per column:")
print(event_logs_clean.isnull().sum())


=== Event Logs - AFTER CLEANING ===
Total records: 2000
Null values per column:
user_id       0
event_type    0
event_time    0
product_id    0
amount        0
dtype: int64


In [79]:
# Save cleaned data
event_logs_clean.to_csv('cleaned_event_logs.csv', index=False)

In [81]:
# ## 2. Trend Report Cleaning
# - **Key Metrics**: `week`, `avg_users`, `sales_growth_rate`

In [83]:
trend_report = pd.read_csv('trend_report.csv')

In [85]:
# Show initial null summary
print("\n=== Trend Report - BEFORE CLEANING ===")
print(f"Total records: {len(trend_report)}")
print("Null values per column:")
print(trend_report.iloc[:, :3].isnull().sum())


=== Trend Report - BEFORE CLEANING ===
Total records: 20
Null values per column:
week                 0
avg_users            0
sales_growth_rate    0
dtype: int64


In [87]:
# Keep only relevant columns
trend_clean = trend_report.iloc[:, :3].copy()
trend_clean.columns = ['week', 'avg_users', 'sales_growth_rate']

In [89]:
# Handle dates - specify ISO format to avoid warnings
trend_clean['week'] = pd.to_datetime(
    trend_clean['week'], 
    format='ISO8601',  # Standard date format
    errors='coerce'
)

In [91]:
# Sort before forward-filling
trend_clean.sort_values('week', inplace=True)

In [93]:
# Fill numeric metrics using forward fill
trend_clean['avg_users'] = trend_clean['avg_users'].fillna(0)
trend_clean['sales_growth_rate'] = trend_clean['sales_growth_rate'].ffill().fillna(0)

In [95]:
# Show post-cleaning summary
print("\n=== Trend Report - AFTER CLEANING ===")
print(f"Total records: {len(trend_clean)}")
print("Null values per column:")
print(trend_clean.isnull().sum())


=== Trend Report - AFTER CLEANING ===
Total records: 20
Null values per column:
week                 20
avg_users             0
sales_growth_rate     0
dtype: int64


In [97]:
trend_clean.to_csv('cleaned_trend_report.csv', index=False)

In [99]:
# ## 3. Marketing Summary Cleaning
# - **Core Columns**: `date`, `users_active`, `total_sales`, `new_customers`

In [101]:
marketing = pd.read_csv('marketing_summary.csv')

In [103]:
# Show initial null summary
print("\n=== Marketing Summary - BEFORE CLEANING ===")
print(f"Total records: {len(marketing)}")
print("Null values per column:")
print(marketing.iloc[:, :4].isnull().sum())


=== Marketing Summary - BEFORE CLEANING ===
Total records: 100
Null values per column:
date             0
users_active     0
total_sales      0
new_customers    0
dtype: int64


In [105]:
# Keep relevant columns
marketing_clean = marketing.iloc[:, :4].copy()
marketing_clean.columns = ['date', 'users_active', 'total_sales', 'new_customers']

In [107]:
# Date conversion - specify format to avoid warnings
marketing_clean['date'] = pd.to_datetime(
    marketing_clean['date'], 
    format='ISO8601',  # Standard date format
    errors='coerce'
)

In [109]:
# Fill sequential data (time-series)
marketing_clean.sort_values('date', inplace=True)
for col in ['users_active', 'total_sales', 'new_customers']:
    marketing_clean[col] = marketing_clean[col].ffill().fillna(0)

In [113]:
# Show post-cleaning summary
print("\n=== Marketing Summary - AFTER CLEANING ===")
print(f"Total records: {len(marketing_clean)}")
print("Null values per column:")
print(marketing_clean.isnull().sum())


=== Marketing Summary - AFTER CLEANING ===
Total records: 100
Null values per column:
date             0
users_active     0
total_sales      0
new_customers    0
dtype: int64


In [115]:
marketing_clean.to_csv('cleaned_marketing_summary.csv', index=False)

In [117]:
# ## Next Steps for Analysis
# With cleaned data, we can now:
# 1. **Identify Peak Order Times**:
# 2. **Correlate System Errors with Order Volume**:
# 3. **Forecast Demand**:
# 4. **Infrastructure Scaling Analysis**: